<a href="https://colab.research.google.com/github/Sarana-F/DAY-1/blob/main/AdvanceFeedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install nltk pandas transformers streamlit twilio googletrans==4.0.0-rc1 emoji

In [3]:
# Import necessary libraries
import nltk
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import pipeline
from googletrans import Translator
import emoji
import matplotlib.pyplot as plt
import streamlit as st
from twilio.rest import Client

In [4]:
# Download NLTK data
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [5]:
# Load a pretrained sentiment analysis model from Transformers
sentiment_pipeline = pipeline('sentiment-analysis')

# Real-time feedback collection table structure (example schema for database)
# CREATE TABLE feedback (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     student_id INT,
#     feedback_text TEXT NOT NULL,
#     timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
# );

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [6]:
# Preprocess text function
def preprocess_text(text):
    # Convert emojis to text
    text = emoji.demojize(text)

    # Tokenize and lower text
    tokens = word_tokenize(text.lower())

    # Remove custom stopwords
    custom_stopwords = set(stopwords.words('english')).union({'class', 'lecture', 'teacher'})
    filtered_tokens = [token for token in tokens if token not in custom_stopwords]

    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # Rejoin processed tokens
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text

In [7]:
# Sentiment classification using BERT model
def get_sentiment(text):
    result = sentiment_pipeline(text)[0]
    return 1 if result['label'] == 'POSITIVE' else 0


In [8]:
# Feedback categorization
categories = {
    'Academic': ['lecture', 'class', 'assignment'],
    'Social': ['group', 'teamwork', 'activity'],
    'Personal': ['stress', 'anxiety', 'confidence']
}

In [9]:
def categorize_feedback(text):
    for category, keywords in categories.items():
        if any(keyword in text for keyword in keywords):
            return category
    return 'Other'

# Translation to English (for multi-language feedback support)
translator = Translator()

def translate_to_english(text):
    return translator.translate(text, src='auto', dest='en').text

# Example dataset (replace with real-time feedback collection later)
data = {
    'reviewText': [
        'The class was amazing!',
        'I felt lost during the lecture.',
        'The teacher explained well.',
        'I feel stressed about exams.'
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Preprocess and analyze feedback
st.title("Real-Time Student Sentiment Dashboard")

# Preprocess text
st.write("**Preprocessing Feedback**")
df['ProcessedText'] = df['reviewText'].apply(preprocess_text)

# Translate to English if needed
df['TranslatedText'] = df['ProcessedText'].apply(translate_to_english)

# Sentiment analysis
df['Sentiment'] = df['TranslatedText'].apply(get_sentiment)

# Categorize feedback
df['Category'] = df['TranslatedText'].apply(categorize_feedback)

# Display processed feedback
df['SentimentLabel'] = df['Sentiment'].map({1: 'Positive', 0: 'Negative'})
st.table(df[['reviewText', 'ProcessedText', 'SentimentLabel', 'Category']])


2025-01-18 15:17:06.771 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 15:17:07.740 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-01-18 15:17:07.743 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 15:17:07.745 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 15:17:07.747 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 15:17:07.750 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 15:17:07.751 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 15:17:15.029 Thread 'MainThread': mi

DeltaGenerator()

In [11]:

# Visualize sentiment distribution
st.write("**Sentiment Distribution**")
plt.figure(figsize=(6, 4))
df['SentimentLabel'].value_counts().plot(kind='bar', color=['green', 'red'])
plt.title("Sentiment Distribution")
st.pyplot(plt)

# Send real-time alerts (example: Twilio)
def send_alert(message):
    try:
        client = Client('ACCOUNT_SID', 'AUTH_TOKEN')
        client.messages.create(
            to="+1234567890",
            from_="+0987654321",
            body=message
        )
    except Exception as e:
        st.error(f"Failed to send alert: {e}")

# Check for critical feedback and notify
critical_feedback = df[df['Category'] == 'Personal']
if not critical_feedback.empty:
    try:
        send_alert("Critical feedback detected: " + critical_feedback.iloc[0]['reviewText'])
    except Exception as e:
        st.error(f"Error while sending alert: {e}")

st.write("Real-time notifications sent for critical feedback.")

# Run the Streamlit app
# streamlit run this_script.py


2025-01-18 15:26:20.018 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 15:26:20.023 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 15:26:20.029 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 15:26:20.033 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 15:26:20.127 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 15:26:20.755 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 15:26:20.758 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 15:26:20.806 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar